In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace,WebSearchTool, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
import os
from pydantic import BaseModel
from agents.model_settings import ModelSettings

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI


In [4]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

### SYSTEM TOOLS

In [5]:
from instructions import websearch_instruction, app_launcher_instruction, file_searcher_instruction, clipboard_manager_instruction

In [6]:
from launch_app import launch_app

In [7]:
APP_LAUNCHER_AGENT=Agent(name="Gemini App laucher", instructions=app_launcher_instruction, model=gemini_model, tools=[launch_app])

In [8]:
# with trace("App launcher"):
#     result = await Runner.run(APP_LAUNCHER_AGENT, "please open chrome")

In [9]:
# print(result.final_output)

In [10]:
from search_file import search_file

In [11]:
FILE_SEARCHER_AGENT=Agent(name="Gemini Folder/File searcher", instructions=file_searcher_instruction, model="gpt-4o-mini", tools=[search_file])

In [12]:
# with trace("File searcher"):
#     result = await Runner.run(FILE_SEARCHER_AGENT, "Can you please tell me where is pdf_data.json file")

In [13]:
# print(result.final_output)

In [14]:
from clipboard_manager import save_clipboard, get_clipboard_history, get_last_clipboard

In [15]:
CLIPBOARD_MANAGER=Agent(name="Gemini  clipboard manager", instructions=clipboard_manager_instruction, model=gemini_model, tools=[save_clipboard, get_clipboard_history, get_last_clipboard])

In [16]:
# with trace("clipboard manager"):
#     result = await Runner.run(CLIPBOARD_MANAGER, "can you please save my clip board history, and show it to me")

### INTERNET TOOLS

In [17]:
from wikipedia_tool import search_wikipedia

In [18]:

SEARCH_AGENT = Agent(
    name="Search agent",
    instructions=websearch_instruction,
    tools=[WebSearchTool(search_context_size="low"),search_wikipedia],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [19]:
# message = "What is AI and how it works"

# with trace("Search"):
#     result = await Runner.run(SEARCH_AGENT, message)
# print(result.final_output)

In [20]:
from docker_runner import run_code_in_docker
from instructions import code_helper_instruction

In [21]:
print(code_helper_instruction)

You are an expert coding assistant with deep knowledge of multiple programming languages, frameworks, and tools. Your task is to generate correct, optimized, and well-documented code in a single attempt based on the user’s request. You are also provided with a tool (Docker) to test your code safely if it is Python or Node.js. Always use print statements (or console.log in JavaScript) to output final results, otherwise the execution will not show anything. Guidelines: 1. Understanding & Accuracy - Read the request carefully before answering. Choose the most suitable programming language, framework, or library unless the user specifies one. Always ensure code is executable without errors. Do not include hidden reasoning or explanations unless asked. 2. Code Quality - Use clean, maintainable, and well-structured code. Apply best practices (naming conventions, modularity, reusability). Handle edge cases where applicable. Include necessary imports, environment setup, or configuration if req

In [22]:

CODING_AGENT = Agent(
    name="coding agent",
    instructions=code_helper_instruction,
    tools=[run_code_in_docker],
    model=gemini_model,
)

In [23]:
# message = 'Write a python program to calculate the first 10,000 terms \
#     of this series, multiplying the total by 4: 1 - 1/3 + 1/5 - 1/7 + ...'

# with trace("CODING_AGENT"):
#     result = await Runner.run(CODING_AGENT, message)
# print(result.final_output)

In [24]:
from instructions import email_instruction
from send_email import send_email

In [25]:
EMAIL_AGENT = Agent(
    name="email agent",
    instructions=email_instruction,
    tools=[send_email],
    model=gemini_model,
)

In [26]:
# message = """Write an email about about the AI in your own words nothing to fancy: Artificial Intelligence (AI) is a broad field of computer science focused on creating machines and software that can perform tasks which typically require human intelligence.

# These tasks include things like:

# Learning from data and past experiences.

# Reasoning to solve problems or make decisions.

# Recognizing patterns (like in images, speech, or text).

# Understanding and responding to language.

# In simple terms, it's about making computers smart in a specific, narrow way.

# A Simple Analogy:
# Think of a regular calculator. You input numbers and an operation (like 2 + 2), and it outputs the answer (4). It follows a fixed, pre-programmed rule.

# An AI system is more like a student. You show it thousands of math problems and their correct answers. Over time, it learns the patterns and rules for itself. Then, when you give it a new math problem it has never seen before, it can use what it learned to predict the correct answer.  
# IMPORTANT:
# and send it to yourzlove56@gmail.com'
# """
# with trace("EMAIL_AGENT"):
#     result = await Runner.run(EMAIL_AGENT, message)
# print(result.final_output)

In [27]:
from instructions import doc_reader_instruction
from extract_text import extract_text

In [28]:
DOCUMENT_AGENT = Agent(
    name="document agent",
    instructions=doc_reader_instruction,
    tools=[extract_text],
    model=gemini_model,
)

In [29]:
# message = "can you please summarize what is written in (C:/Users/talha/Desktop/LLM/agents work/no.2/jarvis_tools.md)"
# with trace("DOCUMENT_AGENT"):
#     result = await Runner.run(DOCUMENT_AGENT, message)
# print(result.final_output)

In [30]:
from api_tester import api_request
from instructions import api_tester_instruction

In [31]:
API_TESTER_AGENT = Agent(
    name="api agent",
    instructions=api_tester_instruction,
    tools=[api_request],
    model=gemini_model,
)

In [32]:
# message = "can you please text this api :https://api.adviceslip.com/advice"
# with trace("API_TESTER_AGENT"):
#     result = await Runner.run(API_TESTER_AGENT, message)
# print(result.final_output)


In [33]:
from system_control import system_control
from instructions import system_control_instruction
from songplay import play_youtube

In [34]:
SYSTEM_CONTROL_AGENT = Agent(
    name="system control agent",
    instructions=system_control_instruction,
    tools=[system_control,play_youtube],
    model='gpt-4o-mini',
)

In [35]:
# message = "can you please increase the volume"
# with trace("SYSTEM_CONTROL_AGENT"):
#     result = await Runner.run(SYSTEM_CONTROL_AGENT, message)
# print(result.final_output)


In [36]:
# message = "can you please play a song Here Comes the Sun "
# with trace("SYSTEM_CONTROL_AGENT"):
#     result = await Runner.run(SYSTEM_CONTROL_AGENT, message)
# print(result.final_output)


In [37]:
from instructions import web_scraper_instruction
from scraper import scrape_dynamic_website

In [38]:
WEB_SCRAPER_AGENT = Agent(
    name="system control agent",
    instructions=web_scraper_instruction,
    tools=[scrape_dynamic_website],
    model=gemini_model,
)

In [39]:
# message = "can you please scrape https://quotes.toscrape.com/js/ and tell me what you got "
# with trace("WEB_SCRAPER_AGENT"):
#     result = await Runner.run(WEB_SCRAPER_AGENT, message)
# print(result.final_output)


In [40]:
from runcommand import run_command
from instructions import powershell_command_runner_instruction

In [41]:
POWERSHELL_AGENT = Agent(
    name="system control agent",
    instructions=powershell_command_runner_instruction,
    tools=[run_command],
    model=gemini_model,
)

In [42]:
message = "can you please run echo Hello from PowerShell!"
with trace("POWERSHELL_AGENT"):
    result = await Runner.run(POWERSHELL_AGENT, message)
print(result.final_output)

Hello from PowerShell!

